<a href="https://colab.research.google.com/github/sainiakhil/RAG-Chat-Bot-Streamlit/blob/main/Standard_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-gpu
!pip install PyPDF2
!pip install bitsandbytes
!pip install sentence-transformers
!pip install transformers
!pip install huggingface_hub

In [4]:
import torch
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import PyPDF2
import os
from transformers import pipeline

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
device = torch.device("cuda:0")

In [5]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [9]:
embedding_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

    # Load the model
llm_model = AutoModelForCausalLM.from_pretrained(
        "meta-llama/Llama-2-7b-chat-hf",
        device_map = device,
        quantization_config = bnb_config,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")


# Text generation pipeline for LLM
generator = pipeline('text-generation', model=llm_model, tokenizer=tokenizer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
pdf_reader = PyPDF2.PdfReader('/content/attention-is-all-you-need-Paper.pdf')
pdf_text = ""

    # Extract text from each page
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    pdf_text += page.extract_text()

# Split the PDF content into chunks for better embedding
doc_chunks = [pdf_text[i:i+500] for i in range(0, len(pdf_text), 500)]


In [11]:
vectors = embedding_model.encode(doc_chunks)
vectors = np.array(vectors, dtype=np.float32)


# Initialize FAISS index
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(vectors))

In [29]:
def retrieve_context(query, top_k=3):
    # Embed the user query
    query_vector = embedding_model.encode([query])

    # Search for top-k relevant documents in FAISS
    distances, indices = index.search(np.array(query_vector, dtype=np.float32), top_k)

    # Retrieve relevant document chunks
    retrieved_docs = [doc_chunks[i] for i in indices[0]]

    return retrieved_docs

def generate_answer(query):

    context = retrieve_context(query)

    # Augment the query with context
    augmented_input = query + "\n\n Context:\n" + "\n".join(context) + "\n\n Final Answer:\n"

    inputs = tokenizer(augmented_input, return_tensors="pt").to('cuda')
    output = llm_model.generate(**inputs, max_new_tokens=500)
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

In [15]:
# Example question to the QA bot
query = "what is multi head attention"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)



Answer: what is multi head attention

 Context:
r this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehe
ion. (right) Multi-Head Attention consists of several
attention layers running in parallel.
query with all keys, divide each bypdk, and apply a softmax function to obtain the weights on the
values.
In practice, we compute the attention function on a set of queries simultaneously, packed together
into a matrix Q. The keys and values are also packed together into matrices KandV. We compute
the matrix of outputs as:
Attention(Q;K;V ) = softmax(QKT
pd

In [16]:
# Example question to the QA bot
query = "what is self attention"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)

Answer: what is self attention

 Context:
r this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehe
n terms of
computational complexity, self-attention layers are faster than recurrent layers when the sequence
lengthnis smaller than the representation dimensionality d, which is most often the case with
sentence representations used by state-of-the-art models in machine translations, such as word-piece
[31] and byte-pair [ 25] representations. To improve computational performance for tasks involving
very long sequences, self-attention could be restrict

In [17]:
# Example question to the QA bot
query = "what is transformers in attention all you need paper"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)

Answer: what is transformers in attention all you need paper

 Context:
tention-based models and plan to apply them to other tasks. We
plan to extend the Transformer to problems involving input and output modalities other than text and
to investigate local, restricted attention mechanisms to efﬁciently handle large inputs and outputs
such as images, audio and video. Making generation less sequential is another research goals of ours.
The code we used to train and evaluate our models is available at https://github.com/
tensorflow/tensor2tensor .
Acknowledgements We a
nsion, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 22, 23, 19].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequence-
aligned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [28].
To the best of our knowledge, however, the Transformer is the ﬁrst transduction mo

In [18]:
# Example question to the QA bot
query = "what is positional encoding in transformers"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)

Answer: what is positional encoding in transformers

 Context:
=r)
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed. There are many choices of positional encodings,
learned and ﬁxed [8].
In this work, we use sine and cosine functions of different frequencies:
PE(pos;2i)=sin(pos=100002i=d model)
PE(pos;2i+1)=cos(pos=100002i=d model)
whereposis the position and iis the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The w
then generates an output
sequence (y1;:::;y m)of symbols one element at a time. At each step the model is auto-regressive
[9], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 En

In [22]:
# Example question to the QA bot
query = "what is YOLOV8"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)

No relevant documents found for the query.
Answer: No relevant information was found to answer your query.


In [33]:
# Example question to the QA bot
query = "what decoder contains in transformers architecture"

# Get the answer using the QA bot
answer = generate_answer(query)
print("Answer:", answer)

Answer: what decoder contains in transformers architecture

 Context:
ilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512 .
Decoder: The decoder is also composed of a stack of N= 6identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head
attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalizat
then generates an output
sequence (y1;:::;y m)of symbols one element at a time. At each step the model is auto-regressive
[9], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.